# Example

This notebooks exemplifies the functionality provided by *GRAPE-traffic*, the estimation of environmental impacts with trajectory data.

It uses the `quickstart` dataset from the *traffic* library, for which no access credentials to the Opensky Network are necessary. Results produces are stored in the path specified in `output_folder` below.


In [1]:
from pathlib import Path

from traffic.data.samples import quickstart

from GRAPE_traffic import AirportTraffic, GrapeTraffic

output_folder = Path("out/example")
output_folder.mkdir(parents=True, exist_ok=True)

c:\Users\gonca\Dev\GRAPE-traffic\.venv\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


## Input Trajectories

We first determine which trajectories in the dataset are arrivals and which are departures, simply by looking at the sign of the mean vertical rate. This is a simplification only used for demonstration purposes.

We will focus on flights at Paris Orly as an example. We set the destination and origin features of arrivals/departures as `LFPO` respectively, and deal later with arrival/departures which are not for Paris Orly

In [2]:
arr = quickstart.feature_lt("vertical_rate_mean", 0).eval()
arr.data["destination"] = "LFPO"  # type: ignore
print(f"{len(arr)} arrivals")  # type: ignore

arr.map_leaflet(airport="LFPO")  # type: ignore

99 arrivals


Map(center=[48.72333, 2.37944], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

In [3]:
dep = quickstart.feature_gt("vertical_rate_mean", 0).eval()
dep.data["origin"] = "LFPO"  # type: ignore
print(f"{len(dep)} departures")  # type: ignore

dep.map_leaflet(airport="LFPO")  # type: ignore

136 departures


Map(center=[48.72333, 2.37944], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

## Trajectory Processing

Having obtained the arrival and departure flights as `Traffic` instances, we can instantiate the `AirportTraffic` instance and start processing. The first steps will be identical for most use cases:

- `clean()`: basic clean up for each flight, such as removing overlapping points (in time), missing data and outliers.
- `assign_ids()`: assign incremental `flight_id` feature to all flights. Arrivals default to `arr_00##` and departures to `dep_00##`.
- `cumulative_distance()`: add the feature `cumdist` and remove overlapping points (geographically).

We then call the `list_all()` method, which returns a pandas DataFrame which contains feature data for each flight, such as runway used, ANP aircraft identified, maximum cumulative distance, ... .

In [4]:
apt_traffic = AirportTraffic(arr, dep).clean().assign_ids().cumulative_distance()

f = apt_traffic.list_all()

if f is not None:
    print(f"No runway: {f['runway'].isna().sum()}")
    print(f"No aircraft: {f['aircraft_id'].isna().sum()}")
    print(f"Cumdist < {AirportTraffic.cumdist_min}: {(f['cumdist_max'] < AirportTraffic.cumdist_min).sum()}")

No runway: 177
No aircraft: 27
Cumdist < 10.0: 2


From the flight feature list it is already clear that some flights do not pass the necessary criteria to be used for environmental impact estimation. The methods used to clean out invalid data as well as to further clean the trajectories are `clean_arrivals()` for arrivals and `clean_departures()` for departures. Check out the documentation of each method to see exactly which actions are performed.

We plot the arrival trajectories which demonstrates the processing done, e.g. arrivals for other airports than Paris Orly were discarded and arrival trajectories are clipped at a certain maximum altitude and at the landing threshold.

In [5]:
apt_traffic = apt_traffic.clean_arrivals().clean_departures()

print(f"Arrivals: {len(apt_traffic.arr)}")  # type: ignore
print(f"Departures: {len(apt_traffic.dep)}")  # type: ignore

apt_traffic.arr.map_leaflet(airport="LFPO")  # type: ignore

Arrivals: 27
Departures: 31


Map(center=[48.72333, 2.37944], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

The last step in trajectory processing is to enhance the trajectory data with all the features necessary to estimate environmental impacts, i.e. TAS and aircraft thrust. The method `compute_thrust()` will add all required features.

Below we also demonstrate the capability of `AirportTraffic` of saving and loading data. This is done simply by storing an `arrivals.csv` and a `departures.csv` file containing the trajectory data in a given folder.

In [6]:
apt_traffic.compute_thrust()

apt_traffic.save(output_folder)
apt_traffic = AirportTraffic.from_folder(output_folder)

## Environmental Impact Calculation

With processed and enhanced trajectories, we can now estimate noise and local air quality emissions with GRAPE. The `GrapeTraffic` class provides the functionality of inserting the data in an `AirportTraffic` instance into a new GRAPE study and run the calculations. The constructor takes three parameters:

- `apt_traffic`: the `AirportTraffic` instance for which to calculate environmental impacts.
- `receptors_path`: a file path to a `.csv` file containing the receptor locations for which to estimate noise. We use the locations in `example receptors.csv`, containing 4 randomly selected locations around Paris Orly.
- `study_path`: the file path at which a new GRAPE study will be created. This defaults to `study.grp` in the working directory, but we set it to `example.grp` in the output folder defined above.

After instantiating `GrapeTraffic`, we call the `run()` method which will take care of all the tasks:

- call the [GRAPE command line tool](https://goncaloroque30.github.io/GRAPE-Docs/Application/CommandLineTool/) to create a new study at the `study_path` specified and import the ANP database into it.
- transform all the input data in the `AirportTraffic` instance into the format specified in the [GRAPE schema](https://goncaloroque30.github.io/GRAPE-Docs/Schema/), especially the sections *LTO Fuel & Emissions*, *Fleet* and *Operations*.
- create a scenario in the study with all flights in the `AirportTraffic` instance, a performance run to estimate fuel flow according to Doc9889, a noise run with the Doc29 noise model and the receptors specified and two emissions runs, one which just outputs the static values in the LTO cycle and other which calculates emissions for each trajectory segment, using the BFFM2 model. See the `Scenarios`, `Performance Run`, `Noise Run` and `Emissions Run` sections in the [GRAPE schema](https://goncaloroque30.github.io/GRAPE-Docs/Schema/).
- cal again the [GRAPE command line tool](https://goncaloroque30.github.io/GRAPE-Docs/Application/CommandLineTool/) to run the performance run, the noise run and the two emissions runs created.


In [7]:
GrapeTraffic.lto_emissions_cutoff = 3000 + 291  # Paris orly elevation is 291ft
GrapeTraffic(
    apt_traffic,
    receptors_path="notebooks/example receptors.csv",
    study_path=output_folder / "example.grp",
).run()

The noise and local air quality emissions results are now available in the output tables of the study as specified in the [GRAPE schema](https://goncaloroque30.github.io/GRAPE-Docs/Schema/) and can be further analyzed.